In [ ]:
from utils import inference_validator, reasoning, feature_engineering, additional_feature_engineering
import pandas as pd
pd.set_option('display.max_rows', None)  # None means unlimited rows
pd.set_option('display.max_columns', None) # None means unlimited columns
pd.set_option('display.width', None)      # None means auto-detect width
pd.set_option('display.max_colwidth', None) # None means unlimited column width

In [ ]:
import pickle, json
with open('saved/nimputer.pkl', 'rb') as f:
    nimputer = pickle.load(f)

with open('saved/cimputer.pkl', 'rb') as f:
    cimputer = pickle.load(f)

with open('saved/label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

    
with open('saved/inference_preprocessor.pkl', 'rb') as f:
    inference_preprocessor = pickle.load(f)

with open('saved/xgb_model.pkl', 'rb') as f:
    xgb = pickle.load(f)

with open('saved/before_feature.json', 'r') as f:
    before_columns = json.load(f)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# Example user input
user_input = {
    'score_3': 0.5, 'score_4': np.nan, 'score_5': 0.7, 'score_6': np.nan,
    'risk_rate': 0.2, 'last_amount_borrowed': 1000, 'last_borrowed_in_months': 12,
    'credit_limit': 5000, 'income': np.nan, 'ok_since': 24, 'n_bankruptcies': 9,
    'n_defaulted_loans': 1, 'n_accounts': 3, 'n_issues': 2,
    'external_data_provider_credit_checks_last_year': 3,
    'external_data_provider_email_seen_before': 1, 'reported_income': 50000,
    'application_time_in_funnel': 5, 'external_data_provider_credit_checks_last_month': 0,
    'external_data_provider_fraud_score': 50, 'shipping_state': np.nan,
    'facebook_profile': True, 'state': np.nan, 'score_1': np.nan, 'score_2': np.nan, "real_state":'a'
}

# user_input = {
#     'score_3': 510, 'score_4': np.nan, 'score_5': 0.480, 'score_6': 92,
#     'risk_rate': 0.45, 'last_amount_borrowed': 600, 'last_borrowed_in_months': 20,
#     'credit_limit': 500, 'income': 60000, 'ok_since': 32, 'n_bankruptcies': 3.0,
#     'n_defaulted_loans': 0, 'n_accounts': 25.0, 'n_issues': 5.0,
#     'external_data_provider_credit_checks_last_year': 4.0,
#     'external_data_provider_email_seen_before': 5, 'reported_income': 50000,
#     'application_time_in_funnel': 900, 'external_data_provider_credit_checks_last_month': 5.0,
#     'external_data_provider_fraud_score': 250, 'shipping_state': np.nan,
#     'facebook_profile': True, 'state': np.nan, 'score_1': '1Rk8w4Ucd5yR3KcqZzLdow==', 'score_2': np.nan, "real_state": np.nan
# }

user_input = {
    "score_3": 500,                     # Min: 0, Max: 990
    "score_4": 80,                    # Min: 86.191572, Max: 113.978234
    "score_5": 0.5,                    # Min: 0.000035, Max: 0.999973
    "score_6": 100,                      # Min: 60.663039, Max: 142.192400
    "risk_rate": 0.6,                  # Min: 0.000000, Max: 0.900000
    "last_amount_borrowed": 10000000,        # Min: 0.000000, Max: 35059.600000
    "last_borrowed_in_months": 1,      # Min: 0.000000, Max: 60.000000
    "credit_limit": 100000,                 # Min: 0.000000, Max: 448269.000000
    "income": 1200000,                   # Min: 4821.180000, Max: 5000028.000000
    "ok_since": 1,                     # Min: 0.000000, Max: 141.000000
    "n_accounts": 1,                 # Min: 0.000000, Max: 49.000000
    "n_issues": 0,                    # Min: 0.000000, Max: 49.000000
    "external_data_provider_credit_checks_last_year": 1, # Min: 0.000000, Max: 1.000000
    "external_data_provider_email_seen_before": 50,        # Min: 0.000000, Max: 59.000000
    "reported_income": 120000,          # Min: 403.000000, Max: 6355500000000000.000000
    "application_time_in_funnel": 300,  # Min: 0.000000, Max: 500.000000
    "external_data_provider_credit_checks_last_month": 0.0, # Min: 0.000000, Max: 3.000000
    "external_data_provider_fraud_score": 0, # Min: 0.000000, Max: 1000.000000
    "shipping_state": np.nan,             # No min/max provided (categorical)
    "facebook_profile": False,          # No min/max provided (boolean)
    "state": "BR-MS",                      # No min/max provided (categorical)
    "score_1": "1Rk8w4Ucd5yR3KcqZzLdow==", # No min/max provided (encoded string)
    "real_state": np.nan                  # No min/max provided (nullnp.nan
}
user_input = inference_validator(user_input)

user_df = pd.DataFrame([user_input])

# Ensure the user input has the same columns as the training data
user_df = user_df.reindex(columns=before_columns, fill_value=np.nan)

# Assuming you know which features each imputer was trained on
c_features = [f for f in cimputer.feature_names_in_ if f in user_df.columns]
n_features = [f for f in nimputer.feature_names_in_ if f in user_df.columns]


# Apply only if features exist
if c_features:
    user_df[c_features] = cimputer.transform(user_df[c_features])
if n_features:
    user_df[n_features] = nimputer.transform(user_df[n_features])

user_df = user_df.drop(columns=["target_default"])
# print("target_default" in user_df.columns)

# Apply the same label encoding to the user input
# for col in c_features:
#     if col in user_df.columns and col != "target_default":
#         # Use the transform method of the fitted LabelEncoder
#         user_df[col] = label_encoders[col].transform(user_df[col])
user_df[[i for i in c_features if i != "target_default"]] = label_encoders.transform(user_df[[i for i in c_features if i != "target_default"]])

# yo_features = [i for i in c_features + n_features if i != "target_default"]
# user_df = user_df.reindex(columns=yo_features)

user_df = feature_engineering(user_df)
user_df = additional_feature_engineering(user_df)
# print(user_df)
# user_processed = inference_preprocessor.transform(user_df)


# Make prediction
prediction = xgb.predict(user_df)
prediction_proba = xgb.predict_proba(user_df)

print("Prediction:", prediction)
print(reasoning[str(prediction[0])])
print("Prediction Probability:", prediction_proba)
# print(user_processed)
# 6 → 3 → 7 → 1 → 2 → 0 → 5 → 4